**POLARITY USING TEXTBLOB**

In [ ]:
import pandas as pd
from textblob import TextBlob

In [ ]:
df = pd.read_csv('./critic_reviews.csv')
df_shuffled = df.sample(frac=1)
random = df_shuffled.head(8000)
random

,Id,rotten_tomatoes_link,critic_name,review_score,review_date,review_content
31534,37927,m/1077077-breakdown,John Paul Powell,4/5,2003-05-07,A well-crafted thriller.
873706,932275,m/the_blackcoats_daughter,Noel Murray,NaN,2017-03-30,"""The Blackcoat's Daughter"" connects the pieces..."
773428,826400,m/small_time_2014,Radheyan Simonpillai,3/5,2014-04-17,"With his personal proximity to the subject, Jo..."
827728,883869,m/stone,James Bradshaw,3/4,2010-09-14,It's worth the price of admission to see Norto...
46469,55712,m/1131148-ned_kelly,Glenn Whipp,2.5/4,2004-03-26,Jordan stages the action scenes with energy an...
...,...,...,...,...,...,...
210109,227323,m/casino_jack_and_the_united_state_of_money,Dan Lybarger,4.5/5,2010-06-05,The subject matter is likely to enrage and for...
126801,139174,m/anthony_jeselnik_thoughts_and_prayers,Brett Smiley,NaN,2019-07-11,"On stage, Jeselnik is an unapologetically offe..."
1045954,1110849,m/wine_country,Jordan Hoffman,83/100,2019-05-09,"By slyly acknowledging the clichés of ""this ty..."
785532,839370,m/son_of_saul,Peter Travers,3.5/4,2015-12-18,A Hungarian masterwork from first-time directo...


In [ ]:
dictionary = []

for text in random['review_content']:
    sentiments = {}
    blob = TextBlob(str(text))
    sentiment = blob.sentiment.polarity
    if sentiment > 0.0:
      sentiments['sentiments'] = 'positive'
    elif sentiment == 0.0:
      sentiments['sentiments'] = 'neutral'
    else:
      sentiments['sentiments'] = 'negative'

    dictionary.append(sentiments)
    print(sentiments)

Streaming output truncated to the last 5000 lines.
{'sentiments': 'negative'}
{'sentiments': 'neutral'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'negative'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'negative'}
{'sentiments': 'negative'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'negative'}
{'sentiments': 'positive'}
{'sentiments': 'negative'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'negative'}
{'sentiments': 'negative'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sentiments': 'positive'}
{'sen

In [ ]:
polarity_textblob = pd.DataFrame(dictionary)
polarity_textblob

,sentiments
0,neutral
1,positive
2,positive
3,positive
4,positive
...,...
7995,positive
7996,negative
7997,positive
7998,negative


In [ ]:
polarity_textblob.to_excel('polarity_textblob.xlsx', index=False)

**POLARITY USING ROBERTA**

In [1]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('critic_reviews.csv')
df_shuffled = df.sample(frac=1)
random = df_shuffled.head(8000)
random

,Id,rotten_tomatoes_link,critic_name,review_score,review_date,review_content
76776,87047,m/21_jump_street_2011,Todd Gilchrist,A,2012-03-05,Lord and Miller have created an instant classi...
96817,107634,m/abominable,Leigh Paatsch,2.5/5,2020-04-20,Viewers under the age of 10 will have few quib...
196143,212707,m/bruno,Owen Gleiberman,A-,2009-07-09,The movie is a toxic dart aimed at the spangly...
863702,922084,m/the-tigger-movie,Staci Layne Wilson,NaN,2005-02-12,"Flouncy, trouncy feel-good fun!"
80580,90982,m/3_days_to_kill,Ryan Syrek,F,2019-08-06,"My least favorite actor is back, and he's look..."
...,...,...,...,...,...,...
337497,363001,m/frozen_2013,Jeffrey M. Anderson,2.5/4,2013-11-27,"It has a beautiful, spunky princess and some v..."
623071,665886,m/preachers_wife,Peter Stack,3/4,2002-06-18,The Preacher's Wife qualifies as old-fashioned...
1026309,1089945,m/wassup_rockers_2006,Matt McKillop,1.5/5,2006-06-27,a bad story poorly told.
730427,780547,m/serendipity,Josh Gilchrist,NaN,2002-12-19,Serendipity director Peter Chelsom gives us ch...


In [3]:
MODEL = f'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
example = random['review_content'].loc[623071]
example

"The Preacher's Wife qualifies as old-fashioned Hollywood movie making at its corniest and most saccharine. But the mostly African American cast puts an engaging spin on the chestnut, and it works beautifully."

In [6]:
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}

print(scores_dict)

{'roberta_neg': 0.013502195, 'roberta_neu': 0.12120604, 'roberta_pos': 0.8652917}


In [8]:
def evaluate_sentiment(row):
    roberta_neg = row['roberta_neg']
    roberta_pos = row['roberta_pos']
    roberta_neu = row['roberta_neu']

    if roberta_neg > roberta_pos and roberta_neg > roberta_neu:
        return 'negative'
    elif roberta_pos > roberta_neg and roberta_pos > roberta_neu:
        return 'positive'
    else:
        return 'neutral'

result = evaluate_sentiment(scores_dict)
print("Sentiment value:", result)

Sentiment value: positive


In [9]:
def polarity_scores_roberta(example):
  encoded_text = tokenizer(example, return_tensors='pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
      'roberta_neg' : scores[0],
      'roberta_neu' : scores[1],
      'roberta_pos' : scores[2]
  }
  return scores_dict

In [10]:
res = {}
for i, row in tqdm(random.iterrows(), total=len(random)):
  try:
    text = row['review_content']
    myid = row['Id']
    roberta_result = polarity_scores_roberta(text)
    res[myid] = roberta_result
  except RuntimeError:
    print(f'Broke for id {myid}')

  0%|          | 0/8000 [00:00<?, ?it/s]

In [11]:
results_df = pd.DataFrame(res).T
results_df

,roberta_neg,roberta_neu,roberta_pos
87047,0.001356,0.019092,0.979552
107634,0.011443,0.188754,0.799803
212707,0.928683,0.066291,0.005026
922084,0.002336,0.032648,0.965016
90982,0.469924,0.362028,0.168048
...,...,...,...
363001,0.001217,0.013065,0.985718
665886,0.013502,0.121206,0.865292
1089945,0.942628,0.053547,0.003825
780547,0.339880,0.543988,0.116132


In [12]:
def evaluate_overall_sentiment(dictionary):
    result_dict = {}
    for key, row in dictionary.items():
        roberta_neg = row.get('roberta_neg')
        roberta_pos = row.get('roberta_pos')
        roberta_neu = row.get('roberta_neu')

        if roberta_neg > roberta_pos and roberta_neg > roberta_neu:
            result_dict[key] = 'negative'
        elif roberta_pos > roberta_neg and roberta_pos > roberta_neu:
            result_dict[key] = 'positive'
        else:
            result_dict[key] = 'neutral'
    return result_dict

overall_result = evaluate_overall_sentiment(res)
print(overall_result)

{87047: 'positive', 107634: 'positive', 212707: 'negative', 922084: 'positive', 90982: 'negative', 782881: 'negative', 157153: 'positive', 547484: 'neutral', 648059: 'negative', 321823: 'positive', 126494: 'neutral', 355949: 'neutral', 455058: 'positive', 109952: 'positive', 698948: 'neutral', 1052523: 'negative', 307616: 'negative', 47905: 'negative', 125613: 'positive', 323698: 'positive', 999852: 'neutral', 949251: 'neutral', 794098: 'negative', 882535: 'positive', 417383: 'positive', 859867: 'positive', 466354: 'negative', 593258: 'negative', 650813: 'neutral', 64655: 'negative', 189472: 'neutral', 766544: 'negative', 834141: 'positive', 803401: 'positive', 592348: 'positive', 192050: 'positive', 881700: 'neutral', 836386: 'neutral', 214951: 'positive', 913764: 'neutral', 274993: 'negative', 173007: 'neutral', 727957: 'negative', 362748: 'positive', 718547: 'positive', 1014469: 'neutral', 1043194: 'positive', 4105: 'neutral', 527980: 'positive', 583253: 'positive', 899031: 'negativ

In [26]:
overall_results_df = pd.DataFrame(overall_result.items(), columns=['Id', 'sentiment'])
overall_results_df.set_index('Id', inplace=True)
overall_results_df.index.name = None
overall_results_df = overall_results_df.reset_index().rename(columns={'index': 'Id'})
overall_results_df

,Id,sentiment
0,87047,positive
1,107634,positive
2,212707,negative
3,922084,positive
4,90982,negative
...,...,...
7995,363001,positive
7996,665886,positive
7997,1089945,negative
7998,780547,neutral


In [27]:
merged_df = pd.merge(random, overall_results_df, on='Id', how='inner')
merged_df

,Id,rotten_tomatoes_link,critic_name,review_score,review_date,review_content,sentiment
0,87047,m/21_jump_street_2011,Todd Gilchrist,A,2012-03-05,Lord and Miller have created an instant classi...,positive
1,107634,m/abominable,Leigh Paatsch,2.5/5,2020-04-20,Viewers under the age of 10 will have few quib...,positive
2,212707,m/bruno,Owen Gleiberman,A-,2009-07-09,The movie is a toxic dart aimed at the spangly...,negative
3,922084,m/the-tigger-movie,Staci Layne Wilson,NaN,2005-02-12,"Flouncy, trouncy feel-good fun!",positive
4,90982,m/3_days_to_kill,Ryan Syrek,F,2019-08-06,"My least favorite actor is back, and he's look...",negative
...,...,...,...,...,...,...,...
7995,363001,m/frozen_2013,Jeffrey M. Anderson,2.5/4,2013-11-27,"It has a beautiful, spunky princess and some v...",positive
7996,665886,m/preachers_wife,Peter Stack,3/4,2002-06-18,The Preacher's Wife qualifies as old-fashioned...,positive
7997,1089945,m/wassup_rockers_2006,Matt McKillop,1.5/5,2006-06-27,a bad story poorly told.,negative
7998,780547,m/serendipity,Josh Gilchrist,NaN,2002-12-19,Serendipity director Peter Chelsom gives us ch...,neutral


In [ ]:
overall_results_df.to_excel('polarity_roberta.xlsx', index=False)